In [2]:

import torch 
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

import numpy as  np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pickle 

with open('/content/drive/MyDrive/CS247/undersampled/labels_undersampled.pkl', 'rb') as fid:
     y_train = pickle.load(fid)

with open("/content/drive/MyDrive/CS247/undersampled/training_emb_undersampled.pkl",'rb') as fid:
    x_train = pickle.load(fid)


with open('drive/MyDrive/CS247/test_data.pkl', 'rb') as fid:
     test_data= pickle.load(fid)




with open("/content/drive/MyDrive/CS247/sen_emb_test.pkl", 'rb') as fid:
    xtest =pickle.load(fid)

In [15]:
nan_value = float("NaN")
test_data.replace("",nan_value,inplace=True)
test_data.dropna(inplace=True)


y_toxic_test= test_data['toxic'].values
y_severe_toxic_test = test_data['severe_toxic'].values
y_obscene_test = test_data['obscene'].values
y_threat_test = test_data['threat'].values
y_insult_test = test_data['insult'].values
y_identity_hate_test = test_data['identity_hate'].values

In [11]:
y_train[:,1].shape

(30495,)

In [4]:
from sklearn.model_selection import cross_validate as cross_validation, ShuffleSplit, cross_val_score, train_test_split, StratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score,roc_curve, f1_score
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt



def classifier_validation(X_train,Y_train,X_test,Y_test):
    report_scores = {}
    roc_auc_scores ={}
    roccurve = {}
    f1_scores = {}
    best_thresholds = {}
    classifiers = [LogisticRegression(max_iter=700,class_weight='balanced'),BernoulliNB()]
    random = [ 0 for _ in range(len(Y_test))]
    r_score = roc_auc_score(Y_test,random)
    r_fpr , r_tpr, thresholds = roc_curve(Y_test,random)
    random_eval = r_score , r_fpr, r_tpr 
    for classifier in (classifiers):
        classifier.fit(X_train, Y_train)
        probs = classifier.predict_proba(X_test)
        roc_auc_scores[classifier.__class__.__name__] = roc_auc_score(Y_test,probs[:,1]) 
        roc_fpr , roc_tpr, threshold= roc_curve(Y_test,probs[:,1])
        gmeans = np.sqrt(roc_tpr * (1-roc_fpr))
        ix = np.argmax(gmeans)
        best_thresholds[classifier.__class__.__name__] = threshold[ix]
        roccurve[classifier.__class__.__name__] = roc_fpr, roc_tpr 
        predictions = classifier.predict(X_test)
        f1_scores[classifier.__class__.__name__] = f1_score(Y_test,predictions)
        report = classification_report(Y_test, predictions)
        report_scores[classifier.__class__.__name__] = report
        # print(classifier.__class__.__name__ + ":")
        # print(report)
        # print('------------------------------------------------------')
    return report_scores,roc_auc_scores, roccurve, random_eval,f1_scores, best_thresholds

def eval_BERT(xtrain,xtest,Train_Y,Test_Y):
  xtrain = np.array(xtrain)
  xtest = np.array(xtest)
  report_scores, aucscores, roccurve, random, f1scores,best_thresholds = classifier_validation(xtrain,Train_Y,xtest,Test_Y) 
  return report_scores, aucscores, roccurve, random, f1scores, best_thresholds

In [23]:
def run_threshold_lr(xtrain,xtest,ytrain,ytest,tr):
    lr = LogisticRegression(max_iter =700,class_weight='balanced')
    lr.fit(xtrain,ytrain)
    probabilities = lr.predict_proba(xtest)
    y_pred = (probabilities[:,1] >= tr).astype(int)
    f1 = f1_score(ytest,y_pred)
    return f1

def run_threshold_nb(xtrain,xtest,ytrain,ytest,tr):
    nb = BernoulliNB()
    nb.fit(xtrain,ytrain)
    probabilities = nb.predict_proba(xtest)
    y_pred = (probabilities[:,1] >= tr).astype(int)
    f1 = f1_score(ytest,y_pred)
    return f1


def calculate_scores(xtrain,xtest,ytrain,ytest,res1,res2,res1_tr,res2_tr,label):
    names = ['LogisticRegression','BernoulliNB']
    _,_,_,_,f1scores,thresholds = eval_BERT(xtrain,xtest,ytrain,ytest)
    lr_tr = thresholds['LogisticRegression']
    nb_tr = thresholds['BernoulliNB']
    lr_tr_score = run_threshold_lr(xtrain,xtest,ytrain,ytest,lr_tr)
    nb_tr_score = run_threshold_nb(xtrain,xtest,ytrain,ytest,nb_tr)
    res1[label] = f1scores[names[0]]
    res2[label] = f1scores[names[1]]
    res1_tr[label] = lr_tr_score
    res2_tr[label] =  nb_tr_score


def final_results(res_lr,res_nb,res_tr_lr,res_tr_nb):
    N = 6
    sum_lr = 0
    sum_nb = 0
    sum_tr_lr = 0
    sum_tr_nb = 0
    for key in res_lr:
         sum_lr += res_lr[key]

    for key in res_nb:
        sum_nb += res_nb[key]

    for key in res_tr_lr:
        sum_tr_lr += res_tr_lr[key]

    for key in res_tr_nb:
        sum_tr_nb += res_tr_nb[key]

    f1average_lr  = sum_lr / N
    f1average_nb  = sum_nb / N
    f1average_tr_lr = sum_tr_lr / N
    f1average_tr_nb = sum_tr_nb / N
    return f1average_lr , f1average_nb, f1average_tr_lr , f1average_tr_nb



def run_all_classifiers():
    results_lr = {}
    results_nb = {}
    results_lr_tr = {}
    results_nb_tr = {}
    all_y_test = [y_toxic_test,y_severe_toxic_test,y_obscene_test,y_threat_test,y_insult_test,y_identity_hate_test]
    labels = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
    for i in range(y_train.shape[1]):
        calculate_scores(x_train,xtest,y_train[:,i],all_y_test[i],results_lr,results_nb,results_lr_tr,results_nb_tr,labels[i])
    
    res1,res2,res3,res4 = final_results(results_lr,results_nb,results_lr_tr,results_nb_tr)
    return res1, res2,res3,res4




    


In [24]:
f1average_lr , f1average_nb, f1average_lr_tr , f1average_nb_tr  = run_all_classifiers()
print('Logistic Regression F1 score : ', f1average_lr)
print('BernoulliNB F1 score : ',f1average_nb)
print('Logistic Regression with threshold F1 score :', f1average_lr_tr )
print('BernoulliNB with threshold  F1 score : ',f1average_nb_tr)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Logistic Regression F1 score :  0.3645859177723321
BernoulliNB F1 score :  0.2646434832576467
Logistic Regression with threshold F1 score : 0.28068262429406104
BernoulliNB with threshold  F1 score :  0.22891765146987775
